In [17]:
import tkinter as tk
from tkinter import filedialog, ttk, scrolledtext
import csv
import smtplib
import os
import time
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from pathlib import Path
import threading

# --- Global Variables ---
df = None
# DELETED: The global SMTP_PASS variable is no longer needed.

# --- Modern Color Scheme ---
COLORS = {
    'primary': '#2563eb',      # Blue
    'primary_dark': '#1d4ed8', # Darker blue
    'secondary': '#10b981',    # Green
    'secondary_dark': '#059669', # Darker green
    'danger': '#ef4444',       # Red
    'danger_dark': '#dc2626',  # Darker red
    'warning': '#f59e0b',      # Orange
    'bg_primary': '#f8fafc',   # Light gray background
    'bg_secondary': '#ffffff', # White
    'bg_dark': '#1e293b',      # Dark background
    'text_primary': '#0f172a', # Dark text
    'text_secondary': '#64748b', # Gray text
    'border': '#e2e8f0',       # Light border
    'success': '#22c55e'       # Success green
}

# --- Style Configuration Functions ---
def configure_styles():
    """Configure modern styles for ttk widgets"""
    style = ttk.Style()
    style.configure('Modern.TButton', background=COLORS['primary'], foreground='white', borderwidth=0, focuscolor='none', padding=(20, 12))
    style.map('Modern.TButton', background=[('active', COLORS['primary_dark']), ('pressed', COLORS['primary_dark'])])
    style.configure('Success.TButton', background=COLORS['secondary'], foreground='white', borderwidth=0, focuscolor='none', padding=(20, 12))
    style.map('Success.TButton', background=[('active', COLORS['secondary_dark']), ('pressed', COLORS['secondary_dark'])])
    style.configure('Danger.TButton', background=COLORS['danger'], foreground='white', borderwidth=0, focuscolor='none', padding=(20, 12))
    style.configure('Modern.TCombobox', selectbackground=COLORS['primary'], fieldbackground='white', borderwidth=1, relief='solid')

def create_custom_frame(parent, bg_color=None):
    """Create a modern styled frame"""
    return tk.Frame(parent, bg=bg_color or COLORS['bg_secondary'], relief='flat', bd=0)

def create_modern_label(parent, text, font_size=10, font_weight='normal', color=None):
    """Create a modern styled label"""
    return tk.Label(parent, text=text, font=('Segoe UI', font_size, font_weight), fg=color or COLORS['text_primary'], bg=COLORS['bg_secondary'])

def create_modern_entry(parent, width=30, show=None):
    """Create a modern styled entry widget"""
    entry = tk.Entry(parent, font=('Segoe UI', 10), width=width, relief='solid', bd=1, highlightthickness=1, highlightcolor=COLORS['primary'], highlightbackground=COLORS['border'], show=show)
    return entry

# --- Main Functions ---
def upload_csv():
    """Handles the CSV file upload and reads the contacts."""
    global df
    filepath = filedialog.askopenfilename(title="Select Contacts CSV", filetypes=[('CSV Files', '*.csv')])
    if not filepath:
        return
    try:
        with open(filepath, newline='', encoding='utf-8-sig') as f:
            reader = csv.DictReader(f)
            df = list(reader)
        if not df:
            update_status("CSV is empty.", "warning")
            return
        update_status(f"✓ Loaded {len(df)} contacts from {os.path.basename(filepath)}", "success")
        log_message(f"Loaded {len(df)} contacts from {os.path.basename(filepath)}")
    except Exception as e:
        update_status(f"✗ Error reading CSV: {e}", "error")
        log_message(f"Error reading CSV: {e}")

def start_sending_emails():
    """
    Runs the email sending process in a separate thread to keep the UI responsive.
    """
    if df is None:
        update_status("Please upload a contacts CSV first.", "error")
        log_message("Attempted to send without loading a CSV.")
        return

    send_button.config(state="disabled")

    smtp_user = smtp_user_entry.get()
    # MODIFIED: Get the password from the new password_entry field in the UI
    smtp_pass = password_entry.get()

    if not smtp_user or not smtp_pass:
        update_status("Email and Password are required.", "error")
        log_message("Attempted to send without credentials.")
        send_button.config(state="normal")
        return

    selected_choice = selected_template.get()
    filename = template_map.get(selected_choice)
    if not filename:
        log_message(f"Error: Could not find a template for '{selected_choice}'")
        update_status("Template not found.", "error")
        send_button.config(state="normal")
        return

    filepath = os.path.join("templates", filename)

    try:
        with open(filepath, "r", encoding="utf-8") as f:
            template_html = f.read()
        log_message(f"Successfully loaded template: {filename}")
    except FileNotFoundError:
        log_message(f"ERROR: The template file was not found at {filepath}")
        update_status("Template file not found.", "error")
        send_button.config(state="normal")
        return

    try:
        log_message("Connecting to SMTP server...")
        update_status("Connecting to server...", "info")

        with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
            smtp.ehlo()
            smtp.starttls()
            # MODIFIED: Use the password from the UI entry field
            smtp.login(smtp_user, smtp_pass)
            log_message("Logged in. Sending messages...")
            update_status("Sending emails...", "info")

            for i, row in enumerate(df, start=1):
                row.setdefault('HR Name', 'Hiring Manager')
                company_name = row.get('Company Name') or 'your organization'
                row['Company'] = company_name
                to_addr = row.get('Email')

                if not to_addr:
                    log_message(f"[{i}/{len(df)}] Skipping row {i}: missing Email")
                    continue

                msg = build_and_send(smtp_user, to_addr, row, template_html)
                smtp.send_message(msg)
                log_message(f"[{i}/{len(df)}] Email sent to {to_addr}")

                progress = (i / len(df)) * 100
                update_status(f"Sending... {i}/{len(df)} ({progress:.1f}%)", "info")
                time.sleep(1.0)

        log_message("\n✓ All emails sent successfully!")
        update_status("✓ All emails sent successfully!", "success")
    except smtplib.SMTPAuthenticationError:
        log_message("\n✗ ERROR: Login failed. Check your email and app password.")
        update_status("✗ Login failed. Check credentials.", "error")
    except Exception as e:
        log_message(f"\n✗ An unexpected error occurred: {e}")
        update_status("✗ An error occurred.", "error")
    finally:
        send_button.config(state="normal")

def build_and_send(from_addr, to_addr, row, template_html):
    """Builds the email message."""
    msg_root = MIMEMultipart('related')
    msg_root['Subject'] = "IIT Madras Placement Invitation for the 2025-2026 Academic Year"
    msg_root['From'] = from_addr
    msg_root['To'] = to_addr
    msg_alt = MIMEMultipart('alternative')
    msg_root.attach(msg_alt)

    plain_text = f"Dear {row.get('HR Name', 'Hiring Manager')},\nGreetings from the Placement Team, IIT Madras..."
    msg_alt.attach(MIMEText(plain_text, 'plain'))

    html_body = template_html.format(**{"HR Name": row['HR Name'], "Company": row['Company']})
    msg_alt.attach(MIMEText(html_body, 'html'))

    for cid, filepath in [('header', 'header_banner.jpg'), ('research', 'research_hexagon.png'), ('campus', 'campus.jpg'), ('logo', 'iit_logo.png')]:
        if Path(filepath).exists():
            with open(filepath, 'rb') as f:
                img = MIMEImage(f.read())
                img.add_header('Content-ID', f'<{cid}>')
                msg_root.attach(img)
    return msg_root

def log_message(message):
    """Adds a message to the log window in the UI."""
    log_text.insert(tk.END, message + "\n")
    log_text.see(tk.END)

def update_status(message, status_type="info"):
    """Update status label."""
    colors = {"success": COLORS['success'], "error": COLORS['danger'], "warning": COLORS['warning'], "info": COLORS['primary']}
    status_label.config(text=message, fg=colors.get(status_type, COLORS['text_primary']))

# DELETED: The entire `getpass` block has been removed from here.

# --- Modern UI Setup ---
root = tk.Tk()
root.title("IIT Madras Email Sender")
root.geometry("900x700")
root.configure(bg=COLORS['bg_primary'])
configure_styles()

header_frame = tk.Frame(root, bg=COLORS['primary'], height=80)
header_frame.pack(fill="x")
header_frame.pack_propagate(False)
tk.Label(header_frame, text="📧 IIT Madras Email Campaign Manager", font=('Segoe UI', 18, 'bold'), fg='white', bg=COLORS['primary']).pack(expand=True)
tk.Label(header_frame, text="Professional Email Distribution System", font=('Segoe UI', 10), fg='#e2e8f0', bg=COLORS['primary']).pack()

main_container = tk.Frame(root, bg=COLORS['bg_primary'])
main_container.pack(fill="both", expand=True, padx=20, pady=20)

config_card = create_custom_frame(main_container)
config_card.pack(fill="x", pady=(0, 20))
create_modern_label(config_card, "⚙️ Configuration", 14, 'bold').pack(anchor="w", padx=20, pady=(20, 10))

email_frame = tk.Frame(config_card, bg=COLORS['bg_secondary'])
email_frame.pack(fill="x", padx=20, pady=10)
create_modern_label(email_frame, "Gmail Address:", 10, 'normal', COLORS['text_secondary']).pack(anchor="w")
smtp_user_entry = create_modern_entry(email_frame, 40)
smtp_user_entry.insert(0, "ch24m029@smail.iitm.ac.in")
smtp_user_entry.pack(fill="x", pady=(5, 0))

# ADDED: Password field UI block
password_frame = tk.Frame(config_card, bg=COLORS['bg_secondary'])
password_frame.pack(fill="x", padx=20, pady=10)
create_modern_label(password_frame, "Gmail App Password:", 10, 'normal', COLORS['text_secondary']).pack(anchor="w")
password_entry = create_modern_entry(password_frame, 40, show="*")
password_entry.pack(fill="x", pady=(5, 0))

template_frame = tk.Frame(config_card, bg=COLORS['bg_secondary'])
template_frame.pack(fill="x", padx=20, pady=(15, 20))
create_modern_label(template_frame, "Email Template:", 10, 'normal', COLORS['text_secondary']).pack(anchor="w")
template_map = {"🎓 IIT Madras Template": "iit_madras_template.html", "🧪 Pharma Company Template": "pharma_template.html"}
selected_template = tk.StringVar()
template_dropdown = ttk.Combobox(template_frame, textvariable=selected_template, values=list(template_map.keys()), state="readonly", font=('Segoe UI', 10), style='Modern.TCombobox')
template_dropdown.pack(fill="x", pady=(5, 0))
template_dropdown.set(list(template_map.keys())[0])

actions_card = create_custom_frame(main_container)
actions_card.pack(fill="x", pady=(0, 20))
create_modern_label(actions_card, "🚀 Actions", 14, 'bold').pack(anchor="w", padx=20, pady=(20, 15))
buttons_frame = tk.Frame(actions_card, bg=COLORS['bg_secondary'])
buttons_frame.pack(fill="x", padx=20, pady=(0, 20))
upload_button = ttk.Button(buttons_frame, text="📁 Upload Contacts CSV", command=upload_csv, style='Modern.TButton')
upload_button.pack(side="left", fill="x", expand=True, padx=(0, 10))
send_button = ttk.Button(buttons_frame, text="📤 Send All Emails", command=lambda: threading.Thread(target=start_sending_emails, daemon=True).start(), style='Success.TButton')
send_button.pack(side="left", fill="x", expand=True)

status_card = create_custom_frame(main_container)
status_card.pack(fill="x", pady=(0, 20))
create_modern_label(status_card, "📊 Status", 14, 'bold').pack(anchor="w", padx=20, pady=(20, 10))
status_frame = tk.Frame(status_card, bg='#f1f5f9', relief='flat', bd=0)
status_frame.pack(fill="x", padx=20, pady=(0, 20))
status_label = tk.Label(status_frame, text="💡 Please upload a CSV file to begin", font=('Segoe UI', 10), fg=COLORS['text_secondary'], bg='#f1f5f9', anchor="w", padx=15, pady=10)
status_label.pack(fill="x")

log_card = create_custom_frame(main_container)
log_card.pack(fill="both", expand=True)
create_modern_label(log_card, "📝 Activity Log", 14, 'bold').pack(anchor="w", padx=20, pady=(20, 10))
log_frame = tk.Frame(log_card, bg=COLORS['bg_secondary'])
log_frame.pack(fill="both", expand=True, padx=20, pady=(0, 20))
log_text = scrolledtext.ScrolledText(log_frame, height=12, wrap=tk.WORD, font=('Consolas', 9), bg='#1e293b', fg='#e2e8f0', relief='flat', bd=0, padx=10, pady=10)
log_text.pack(fill="both", expand=True)
log_message("🔧 Email sender initialized successfully")
log_message("📋 Ready to load contacts and send emails")

footer_frame = tk.Frame(root, bg=COLORS['bg_primary'], height=30)
footer_frame.pack(fill="x")
footer_frame.pack_propagate(False)
tk.Label(footer_frame, text="© 2024 IIT Madras Placement Team | Built with ❤️ for efficient communication", font=('Segoe UI', 8), fg=COLORS['text_secondary'], bg=COLORS['bg_primary']).pack(expand=True)

root.mainloop()